# Development Environment and Permissions


_**Use at least a `t3.large` instance otherwise preprocessing will take ages.**_

## Installation

_*Note:* we only install the required libraries from Hugging Face and AWS. You also need PyTorch or Tensorflow, if not already installed_

In [ ]:
!pip install sagemaker boto3 torch transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
  

## Permissions

_If you are going to use Sagemaker in a local environment, you need access to an IAM Role with the required permissions for Sagemaker. You can find out more about this [here](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html)_

In [ ]:
import os
os.environ["AWS_ACCESS_KEY_ID"] = ""
os.environ["AWS_SECRET_ACCESS_KEY"] = ""
os.environ["AWS_DEFAULT_REGION"] = "us-east-1"  # Change to your AWS region

In [ ]:
import boto3
import sagemaker

aws_region = "us-east-1"  # Set the correct AWS region

# Manually set the correct role ARN (replace with your actual ARN)
role_arn = ""

# Create a SageMaker session
boto_session = boto3.Session(region_name=aws_region)
sess = sagemaker.Session(boto_session=boto_session)

print(f"SageMaker Role ARN: {role_arn}")
print(f"SageMaker Default Bucket: {sess.default_bucket()}")
print(f"SageMaker Session Region: {sess.boto_region_name}")

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
SageMaker Role ARN: arn:aws:iam::437552477415:role/sageMaker
SageMaker Default Bucket: sagemaker-us-east-1-437552477415
SageMaker Session Region: us-east-1


In [ ]:
# Test connection by listing existing SageMaker endpoints
sagemaker_client = boto3.client("sagemaker", region_name=aws_region)

try:
    response = sagemaker_client.list_endpoints(MaxResults=5)
    print("✅ Successfully connected to SageMaker!")
    print("SageMaker Endpoints:", response["Endpoints"])
except Exception as e:
    print("❌ Failed to connect to SageMaker. Error:", str(e))


✅ Successfully connected to SageMaker!
SageMaker Endpoints: [{'EndpointName': 'huggingface-pytorch-inference-2025-01-30-22-48-14-753', 'EndpointArn': 'arn:aws:sagemaker:us-east-1:437552477415:endpoint/huggingface-pytorch-inference-2025-01-30-22-48-14-753', 'CreationTime': datetime.datetime(2025, 1, 30, 22, 48, 15, 365000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2025, 1, 30, 22, 51, 27, 236000, tzinfo=tzlocal()), 'EndpointStatus': 'InService'}, {'EndpointName': 'huggingface-pytorch-inference-2025-01-30-22-46-50-835', 'EndpointArn': 'arn:aws:sagemaker:us-east-1:437552477415:endpoint/huggingface-pytorch-inference-2025-01-30-22-46-50-835', 'CreationTime': datetime.datetime(2025, 1, 30, 22, 46, 51, 588000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2025, 1, 30, 22, 50, 3, 196000, tzinfo=tzlocal()), 'EndpointStatus': 'InService'}]


In [ ]:
import boto3

# Initialize SageMaker client
sagemaker_client = boto3.client("sagemaker", region_name="us-east-1")

# List all deployed endpoints
endpoints = sagemaker_client.list_endpoints(MaxResults=5)

for ep in endpoints["Endpoints"]:
    endpoint_name = ep["EndpointName"]
    endpoint_desc = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)

    # Handle missing 'ModelName' gracefully
    model_name = endpoint_desc.get("ProductionVariants", [{}])[0].get("ModelName", "Unknown")

    print(f"🔹 Endpoint: {endpoint_name} is running model: {model_name}")

    # Only describe model if model_name is valid
    if model_name != "Unknown":
        model_details = sagemaker_client.describe_model(ModelName=model_name)
        env_vars = model_details["PrimaryContainer"].get("Environment", {})

        print(f"   ➡️ HF_MODEL_ID: {env_vars.get('HF_MODEL_ID', 'Unknown')}")


🔹 Endpoint: huggingface-pytorch-inference-2025-01-31-00-17-37-220 is running model: Unknown
🔹 Endpoint: huggingface-pytorch-inference-2025-01-30-22-48-14-753 is running model: Unknown
🔹 Endpoint: huggingface-pytorch-inference-2025-01-30-22-46-50-835 is running model: Unknown


In [ ]:
import boto3
import json

# Define the SageMaker runtime client
sagemaker_runtime = boto3.client("sagemaker-runtime", region_name="us-east-1")

# Specify the endpoint name (choose one from the list)
endpoint_name = "huggingface-pytorch-inference-2025-01-30-22-48-14-753"

# Define input data (adjust based on the model task)
payload = json.dumps({
    "inputs": "The capital of France is [MASK]."
})

# Invoke the endpoint
response = sagemaker_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Body=payload
)

# Process the response
result = json.loads(response["Body"].read().decode())
print("🔍 Model Response:", result)


🔍 Model Response: [{'score': 0.1426878571510315, 'token': 16766, 'token_str': 'marseille', 'sequence': 'the capital of france is marseille.'}, {'score': 0.09020467847585678, 'token': 25387, 'token_str': 'nantes', 'sequence': 'the capital of france is nantes.'}, {'score': 0.08808308094739914, 'token': 17209, 'token_str': 'toulouse', 'sequence': 'the capital of france is toulouse.'}, {'score': 0.08617904037237167, 'token': 3000, 'token_str': 'paris', 'sequence': 'the capital of france is paris.'}, {'score': 0.07720649987459183, 'token': 10241, 'token_str': 'lyon', 'sequence': 'the capital of france is lyon.'}]


In [ ]:
!pip install bitsandbytes accelerate transformers

In [ ]:
!pip install -U bitsandbytes

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Use a valid Qwen or DeepSeek model
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"  # Change if needed

# Load tokenizer & model with forced CPU
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,  # FP16 for better efficiency
    device_map={"": "cpu"}  # Force CPU mode
)

print(f"✅ Model '{model_name}' instantiated successfully in CPU mode!")


model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

✅ Model 'deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B' instantiated successfully in CPU mode!


In [ ]:
import torch

# Define input prompt
input_text = "Write a Python function to calculate the factorial of a number."

# Tokenize input
inputs = tokenizer(input_text, return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")

# Generate response from the model
output = model.generate(**inputs, max_length=200)

# Decode the output
decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

print("🔍 Model Output:\n", decoded_output)


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 0>:10                                                                             │
│                                                                                                  │
│ /usr/local/lib/python3.11/dist-packages/torch/utils/_contextlib.py:116 in decorate_context       │
│                                                                                                  │
│   113 │   @functools.wraps(func)                                                                 │
│   114 │   def decorate_context(*args, **kwargs):                                                 │
│   115 │   │   with ctx_factory():                                                                │
│ ❱ 116 │   │   │   return func(*args, **kwargs)                                                   │
│   117 │                                                                                          │
│   118 │   return decorate_context                                                                │
│   119                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:2252 in generate        │
│                                                                                                  │
│   2249 │   │   │   )                                                                             │
│   2250 │   │   │                                                                                 │
│   2251 │   │   │   # 12. run sample (it degenerates to greedy search when `generation_config.do  │
│ ❱ 2252 │   │   │   result = self._sample(                                                        │
│   2253 │   │   │   │   input_ids,                                                                │
│   2254 │   │   │   │   logits_processor=prepared_logits_processor,                               │
│   2255 │   │   │   │   stopping_criteria=prepared_stopping_criteria,                             │
│                                                                                                  │
│ /usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:3254 in _sample         │
│                                                                                                  │
│   3251 │   │   │   │   outputs = self(**model_inputs, return_dict=True)                          │
│   3252 │   │   │   │   is_prefill = False                                                        │
│   3253 │   │   │   else:                                                                         │
│ ❱ 3254 │   │   │   │   outputs = model_forward(**model_inputs, return_dict=True)                 │
│   3255 │   │   │                                                                                 │
│   3256 │   │   │   # synced_gpus: don't waste resources running the code we don't need; kwargs   │
│   3257 │   │   │   model_kwargs = self._update_model_kwargs_for_generation(                      │
│                                                                                                  │
│ /usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1736 in _wrapped_call_impl    │
│                                                                                                  │
│   1733 │   │   if self._compiled_call_impl is not None:                                          │
│   1734 │   │   │   return self._compiled_call_impl(*args, **kwargs)  # type: ignore[misc]        │
│   1735 │   │   else:                                                                             │
│ ❱ 1736 │   │   │   return self._call_impl(*args, **kwargs)                                       │
│   1737 │                                                   

In [ ]:
# from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri
# from string import ascii_uppercase
# from random import choice

# def generate_random_label():
#     letters = ascii_uppercase
#     return ''.join(choice(letters) for i in range(10))

In [ ]:
# image_uri = get_huggingface_llm_image_uri(
#   backend="huggingface",
#   region=aws_region,
#   version="1.1.0"
# )

# model_name = "MistralLite-" + generate_random_label()

In [ ]:
# image_uri

'763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-tgi-inference:2.0.1-tgi1.1.0-gpu-py39-cu118-ubuntu20.04'

In [ ]:
#  model_name

'MistralLite-DRJIANCKKB'

In [ ]:
# hub_env = {
#   'HF_MODEL_ID': 'amazon/MistralLite',
#   'HF_TASK': 'text-generation',
#   'SM_NUM_GPUS': '1',
#   "MAX_INPUT_LENGTH": '16000',
#   "MAX_TOTAL_TOKENS": '16384',
#   "MAX_BATCH_PREFILL_TOKENS": '16384',
#   "MAX_BATCH_TOTAL_TOKENS":  '16384',
# }

In [ ]:
# model = HuggingFaceModel(
#     name=model_name,
#     env=hub_env,
#     role=role,
#     image_uri=image_uri
# )

In [ ]:
# predictor = model.deploy(
#   initial_instance_count=1,
#   instance_type="ml.g5.2xlarge",
#   endpoint_name=model_name,
# )

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 0>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.11/dist-packages/sagemaker/huggingface/model.py:320 in deploy             │
│                                                                                                  │
│   317 │   │   │   │   inference_tool=inference_tool,                                             │
│   318 │   │   │   )                                                                              │
│   319 │   │                                                                                      │
│ ❱ 320 │   │   return super(HuggingFaceModel, self).deploy(                                       │
│   321 │   │   │   initial_instance_count,                                                        │
│   322 │   │   │   instance_type,                                                                 │
│   323 │   │   │   serializer,                                                                    │
│                                                                                                  │
│ /usr/local/lib/python3.11/dist-packages/sagemaker/model.py:1725 in deploy                        │
│                                                                                                  │
│   1722 │   │   │   return None                                                                   │
│   1723 │   │                                                                                     │
│   1724 │   │   else:  # existing single model endpoint path                                      │
│ ❱ 1725 │   │   │   self._create_sagemaker_model(                                                 │
│   1726 │   │   │   │   instance_type=instance_type,                                              │
│   1727 │   │   │   │   accelerator_type=accelerator_type,                                        │
│   1728 │   │   │   │   tags=tags,                                                                │
│                                                                                                  │
│ /usr/local/lib/python3.11/dist-packages/sagemaker/model.py:985 in _create_sagemaker_model        │
│                                                                                                  │
│    982 │   │   │   │   enable_network_isolation=self._enable_network_isolation,                  │
│    983 │   │   │   │   tags=format_tags(tags),                                                   │
│    984 │   │   │   )                                                                             │
│ ❱  985 │   │   │   self.sagemaker_session.create_model(**create_model_args)                      │
│    986 │                                                                                         │
│    987 │   def _get_model_uri(self):                                                             │
│    988 │   │   model_uri = None                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.11/dist-packages/sagemaker/session.py:4109 in create_model                │
│                                                                                                  │
│   4106 │   │   │   │   else:                                                                     │
│   4107 │   │   │   │   │   raise                                                                 │
│   4108 │   │                                                                                     │
│ ❱ 4109 │   │   self._intercept_create_request(create_model_request, submit, self.create_model._  │
│   4110 │   │   return name                                 

In [ ]:
# List all SageMaker endpoints
endpoints = sagemaker_client.list_endpoints(MaxResults=10)

print("🔍 Active Endpoints:")
for ep in endpoints["Endpoints"]:
    print(f"🟢 {ep['EndpointName']} - Status: {ep['EndpointStatus']}")


🔍 Active Endpoints:


In [ ]:
# Loop through endpoints and delete any with "huggingface" in the name
for ep in endpoints["Endpoints"]:
    endpoint_name = ep["EndpointName"]
    if "huggingface" in endpoint_name.lower():  # Only delete Hugging Face endpoints
        sagemaker_client.delete_endpoint(EndpointName=endpoint_name)
        print(f"🛑 Deleted Endpoint: {endpoint_name}")


🛑 Deleted Endpoint: huggingface-pytorch-inference-2025-01-31-00-17-37-220
🛑 Deleted Endpoint: huggingface-pytorch-inference-2025-01-30-22-48-14-753
🛑 Deleted Endpoint: huggingface-pytorch-inference-2025-01-30-22-46-50-835


In [ ]:
from sagemaker.huggingface.model import HuggingFaceModel
import sagemaker

# Define a lightweight Hugging Face model (DistilBERT for text classification)
hub = {
    "HF_MODEL_ID": "distilbert-base-uncased",  # Lighter model for testing
    "HF_TASK": "fill-mask"  # Simple NLP task
}

# Use the manually set role ARN
role_arn = "arn:aws:iam::437552477415:role/sageMaker"

# Initialize SageMaker session
sess = sagemaker.Session()

# Create a Hugging Face model in SageMaker
huggingface_model = HuggingFaceModel(
    env=hub,
    role=role_arn,
    transformers_version="4.26",
    pytorch_version="1.13",
    py_version="py39",
)

# Deploy the model using a lighter instance type (CPU or small GPU)
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",  # Lighter CPU instance
)

# Test the model
response = predictor.predict({
    "inputs": "The capital of France is [MASK]."
})

print("Model Response:", response)


------!Model Response: [{'score': 0.1426878571510315, 'token': 16766, 'token_str': 'marseille', 'sequence': 'the capital of france is marseille.'}, {'score': 0.09020467847585678, 'token': 25387, 'token_str': 'nantes', 'sequence': 'the capital of france is nantes.'}, {'score': 0.08808308094739914, 'token': 17209, 'token_str': 'toulouse', 'sequence': 'the capital of france is toulouse.'}, {'score': 0.08617904037237167, 'token': 3000, 'token_str': 'paris', 'sequence': 'the capital of france is paris.'}, {'score': 0.07720649987459183, 'token': 10241, 'token_str': 'lyon', 'sequence': 'the capital of france is lyon.'}]


In [ ]:
from sagemaker.huggingface.model import HuggingFaceModel
import sagemaker

# Define DeepSeek model configuration
hub = {
    "HF_MODEL_ID": "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",  # Smaller model
    "HF_TASK": "text-generation"
}


# Use the manually set role ARN
role_arn = "arn:aws:iam::437552477415:role/sageMaker"

# Initialize SageMaker session
sess = sagemaker.Session()

# Create a Hugging Face model in SageMaker
huggingface_model = HuggingFaceModel(
    env=hub,
    role=role_arn,
    transformers_version="4.26",
    pytorch_version="1.13",
    py_version="py39",
    # container_startup_health_check_timeout=600  # Increase timeout to 10 minutes
)

# Deploy the DeepSeek model on a GPU instance (adjust as needed)
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",  # Use a GPU instance
)

# Test the deployed DeepSeek model
response = predictor.predict({
    "inputs": "Explain reinforcement learning in simple terms."
})

print("🔍 Model Response:", response)

---------!

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 0>:34                                                                             │
│                                                                                                  │
│ /usr/local/lib/python3.11/dist-packages/sagemaker/base_predictor.py:212 in predict               │
│                                                                                                  │
│   209 │   │   if inference_component_name:                                                       │
│   210 │   │   │   request_args["InferenceComponentName"] = inference_component_name              │
│   211 │   │                                                                                      │
│ ❱ 212 │   │   response = self.sagemaker_session.sagemaker_runtime_client.invoke_endpoint(**req   │
│   213 │   │   return self._handle_response(response)                                             │
│   214 │                                                                                          │
│   215 │   def _handle_response(self, response):                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.11/dist-packages/botocore/client.py:569 in _api_call                      │
│                                                                                                  │
│    566 │   │   │   │   │   f"{py_operation_name}() only accepts keyword arguments."              │
│    567 │   │   │   │   )                                                                         │
│    568 │   │   │   # The "self" in this scope is referring to the BaseClient.                    │
│ ❱  569 │   │   │   return self._make_api_call(operation_name, kwargs)                            │
│    570 │   │                                                                                     │
│    571 │   │   _api_call.__name__ = str(py_operation_name)                                       │
│    572                                                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.11/dist-packages/botocore/client.py:1023 in _make_api_call                │
│                                                                                                  │
│   1020 │   │   │   │   "Code"                                                                    │
│   1021 │   │   │   )                                                                             │
│   1022 │   │   │   error_class = self.exceptions.from_code(error_code)                           │
│ ❱ 1023 │   │   │   raise error_class(parsed_response, operation_name)                            │
│   1024 │   │   else:                                                                             │
│   1025 │   │   │   return parsed_response                                                        │
│   1026                                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (400) 
from primary with message "{
  "code": 400,
  "type": "InternalServerException",
  "message": "\u0027qwen2\u0027"
}
". See 
https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpo
ints/huggingface-pytorch-inference-2025-01-31-01-54-03-190 in account 437552477415 for more information.